# GPT-OSS 20B Evaluation on DSL List Dataset

This notebook evaluates OpenAI's gpt-oss-20b model on the dsl list dataset using **Execution Prediction**.

**Methodology**: Instead of generating code, the model is given a Python program and predicts its output for a given input (following the paper's approach).

**Requirements**:
- Free Google Colab (T4 GPU)
- HuggingFace account (to load dataset)

**Author**: Code Reasoning Reproduction Team  
**Date**: 2025

## Step 1: Setup Environment

Install required packages for mxfp4 quantization support.

In [ ]:
# Install bleeding-edge PyTorch and transformers
!pip install -q --upgrade torch
!pip install -q transformers triton==3.4 kernels
!pip uninstall -q torchvision torchaudio -y

# Install datasets library
!pip install -q datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.7/899.7 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 47.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 MB 30.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 954.8/954.8 kB 68.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.1/193.1 MB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 75.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.6/63.6 MB 40.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.5/267.5 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.2/288.2 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.3/39.3 MB 57.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170

⚠️ **IMPORTANT**: Please restart your Colab runtime after running the cell above.

Click: **Runtime → Restart runtime**

## Step 2: Load GPT-OSS 20B Model

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

print("Loading gpt-oss-20b model...")
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"CUDA device: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'None'}")

model_id = "openai/gpt-oss-20b"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype="auto",
    device_map="cuda",
)

print("✓ Model loaded successfully!")

Loading gpt-oss-20b model...
CUDA available: True
CUDA device: NVIDIA A100-SXM4-40GB


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/27.9M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/98.0 [00:00<?, ?B/s]

...

_topk_forward.py: 0.00B [00:00, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Fetching 41 files:   0%|          | 0/41 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/177 [00:00<?, ?B/s]

✓ Model loaded successfully!


## Step 3: Load DSL Dataset from Local JSONL

In [ ]:
from datasets import load_dataset
import json

DATASET_REPO_ID = "kuhoos/DSL-List"

print(f"Loading dataset from {DATASET_REPO_ID}...")
raw_dataset = load_dataset(DATASET_REPO_ID)['train']

usable_records = []
for record in raw_dataset:
    inputs = record.get('input')
    outputs = record.get('output')
    mutated_outputs = record.get('mutated_output')
    mutated_code = record.get('mutated_code')

    if not mutated_code or mutated_outputs is None or outputs is None:
        continue

    if isinstance(inputs, str):
        inputs = json.loads(inputs)
    if isinstance(outputs, str):
        outputs = json.loads(outputs)
    if isinstance(mutated_outputs, str):
        mutated_outputs = json.loads(mutated_outputs)

    if not (isinstance(inputs, list) and isinstance(outputs, list) and isinstance(mutated_outputs, list)):
        continue
    if not (len(inputs) == len(outputs) == len(mutated_outputs) == 3):
        continue

    if all(o == m for o, m in zip(outputs, mutated_outputs)):
        continue

    inputs = [json.dumps(i) if not isinstance(i, str) else i for i in inputs]
    outputs = [json.dumps(o) if not isinstance(o, str) else o for o in outputs]
    mutated_outputs = [json.dumps(o) if not isinstance(o, str) else o for o in mutated_outputs]

    usable_records.append({
        'id': record['id'],
        'dsl': record.get('dsl', ''),
        'function_name': record['function_name'],
        'code': record['code'],
        'mutated_code': mutated_code,
        'input': inputs,
        'output': outputs,
        'mutated_output': mutated_outputs,
        'has_mutation': True,
    })

print(f"Usable mutated problems: {len(usable_records)}")

from datasets import Dataset, DatasetDict

dataset = DatasetDict({'train': Dataset.from_list(usable_records)})

print("✓ Dataset loaded and filtered!")
print(f"Total samples: {len(dataset['train'])}")

sample = dataset['train'][0]
print(f"First sample: {sample['id']}")
print(f"Function name: {sample['function_name']}")
for i in range(3):
    print(f"  Case {i+1} input: {sample['input'][i]}")
    print(f"           output: {sample['output'][i]}")


Loading dataset from kuhoos/DSL-List...


programs.jsonl: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/100 [00:00<?, ? examples/s]

Usable mutated problems: 100
✓ Dataset loaded and filtered!
Total samples: 100
First sample: dsl_neural_0
Function name: dsl_prog_0
  Case 1 input: dsl_prog_0(a1=[0, 0, 0, 5])
           output: [0, 0, 5]
  Case 2 input: dsl_prog_0(a1=[1, 0, 2])
           output: [0, 2]
  Case 3 input: dsl_prog_0(a1=[4, 3, 2, 3])
           output: [3, 2, 3]


## Step 4: Define Helper Functions

In [ ]:
import json
import re
import ast
from typing import Dict, Tuple, Optional

def build_execution_prediction_prompt(sample: Dict, case_index:int, use_mutated: bool = False) -> str:
    """
    Build execution prediction prompt for either the original or mutated program.
    """
    function_name = sample['function_name']
    code_key = 'mutated_code' if use_mutated else 'code'
    program = sample.get(code_key)
    test_input = sample['input'][case_index]

    if test_input and test_input.startswith(f"{function_name}(") and test_input.endswith(")"):
        input_args = test_input[len(function_name) + 1:-1]
    else:
        input_args = test_input

    prompt = f"""You are given a Python program and an assertion containing an input to a function. Replace the ?? in the assertion with a literal (no unsimplified expressions, no function calls) representing the function's return value for the given input. Execute the program exactly as written, even if it is incorrect or incomplete. For your final answer, provide the full assertion in [ANSWER] and [/ANSWER] tags.

[PYTHON]
{program}
assert {function_name}({input_args}) == ??
[/PYTHON]"""

    return prompt


def build_execution_choice_prompt(sample: Dict, original_first: bool = True, case_index: int = 0) -> Tuple[str, Dict[str, str]]:
      """Build the execution-choice prompt and map A/B to original/mutated."""
      function_name = sample['function_name']
      original_code = sample['code']
      mutated_code = sample.get('mutated_code') or original_code
      test_input = sample['input'][case_index]

      if test_input and test_input.startswith(f"{function_name}(") and test_input.endswith(")"):
          input_args = test_input[len(function_name) + 1:-1]
      else:
          input_args = test_input

      if original_first:
          program_a, program_b = original_code, mutated_code
          mapping = {'A': 'original', 'B': 'mutated'}
      else:
          program_a, program_b = mutated_code, original_code
          mapping = {'A': 'mutated', 'B': 'original'}

      prompt = (
          "You are given two Python programs, PROGRAM_A and PROGRAM_B, and an assertion. "
          "Your task is to choose which program (A or B) produces the correct return value "
          "for the given assertion input. Then, replace the ?? in the assertion with the "
          "literal return value of the chosen program. Execute the chosen program exactly "
          "as written. Your final answer must be a JSON dictionary of the form:\n\n"
          "{\n"
          '  "chosen_program": "A" or "B",\n'
          '  "assertion": "full_assertion"\n'
          "}\n\n"
          f"[PROGRAM_A]\n{program_a}\n[/PROGRAM_A]\n"
          f"[PROGRAM_B]\n{program_b}\n[/PROGRAM_B]\n"
          f"[ASSERTION]\nassert {function_name}({input_args}) == ??\n[/ASSERTION]"
      )


      return prompt, mapping


def parse_execution_choice_response(response: str) -> Dict[str, str]:
    """
    Parse the model response for execution choice into a structured dictionary.
    Expects a JSON blob containing "chosen_program" and "assertion" keys.
    """
    json_match = re.search(r'\{\s*"chosen_program"\s*:.*?\}', response, re.DOTALL)
    if not json_match:
        raise ValueError('Could not find JSON payload in the response.')

    json_text = json_match.group(0)
    try:
        return json.loads(json_text)
    except json.JSONDecodeError:
        chosen_match = re.search(r'"chosen_program"\s*:\s*"?([A-Za-z])"?', json_text)
        assertion_match = re.search(r'"assertion"\s*:\s*("(?:[^"\]|\.)*")', json_text)
        if not chosen_match or not assertion_match:
            raise ValueError('Failed to parse execution choice JSON response.')

        chosen_program = chosen_match.group(1)
        assertion_literal = assertion_match.group(1)
        try:
            assertion = ast.literal_eval(assertion_literal)
        except Exception:
            assertion = assertion_literal.strip('"')

        return {
            'chosen_program': chosen_program,
            'assertion': assertion,
        }


def extract_output_from_assertion(assertion: str) -> str:
    """Extract the predicted output value from an assertion string."""
    if not assertion:
        return ''

    text = assertion.strip()
    text = re.sub(r'^\[ASSERTION\]\s*', '', text, flags=re.IGNORECASE)
    text = re.sub(r'\s*\[/ASSERTION\]$', '', text, flags=re.IGNORECASE)

    match = re.search(r'assert\s+[\w\.]+\([^)]*\)\s*==\s*(.+)', text)
    if match:
        return match.group(1).strip()

    return text


def extract_answer_from_response(response: str) -> str:
    """
    Extract predicted answer from [ANSWER] tags.

    Expected format: [ANSWER] assert function_name(input) == output [/ANSWER]
    """
    pattern = r'\[ANSWER\](.*?)\[/ANSWER\]'
    matches = re.findall(pattern, response, re.DOTALL | re.IGNORECASE)
    if matches:
        assertion = matches[0].strip()
        match = re.search(r'assert\s+\w+\([^)]*\)\s*==\s*(.+)', assertion)
        if match:
            return match.group(1).strip()
        return assertion

    pattern = r'assert\s+\w+\([^)]*\)\s*==\s*(.+?)(?:|$)'
    matches = re.findall(pattern, response, re.MULTILINE)
    if matches:
        return matches[0].strip()

    return response.strip()


def check_predicted_output(predicted_output: str, expected_output: str) -> Tuple[bool, str]:
    """
    Compare predicted output with expected output.
    """
    try:
        predicted = (predicted_output or '').strip()
        expected = (expected_output or '').strip()

        if predicted == expected:
            return True, None

        try:
            predicted_val = ast.literal_eval(predicted)
            expected_val = ast.literal_eval(expected)
            if predicted_val == expected_val:
                return True, None
        except (ValueError, SyntaxError):
            pass

        return False, f"Predicted: {predicted}, Expected: {expected}"
    except Exception as exc:
        return False, str(exc)


def is_boolean_output(value: str) -> bool:
    """
    Determine whether a serialized output represents a Boolean value.
    """
    if value is None:
        return False

    try:
        parsed = ast.literal_eval(value.strip())
        return isinstance(parsed, bool)
    except (ValueError, SyntaxError, AttributeError):
        lowered = value.strip().lower()
        return lowered in {'true', 'false'}

def _get_output_list(sample, key):
    value = sample.get(key)
    if isinstance(value, list):
        return value
    return [value]
print('✓ Helper functions defined (execution prediction + metrics)')


✓ Helper functions defined (execution prediction + metrics)


## Step 5: Test Execution Prediction on One Sample

This cell demonstrates the **Execution Prediction** task:
1. Model receives a Python program and an assertion with `??`
2. Model predicts what the output will be
3. Model provides answer in `[ANSWER]...[/ANSWER]` tags

In [ ]:
sample = dataset['train'][0]

print(f"Evaluating sample: {sample['id']}")

for case_index in range(len(sample['input'])):
    print(f"=== Case {case_index + 1} ===")
    original_prompt = build_execution_prediction_prompt(sample, use_mutated=False, case_index=case_index)
    print("Prompt:", original_prompt)

    messages = [{"role": "user", "content": original_prompt}]
    inputs = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        return_tensors="pt",
        return_dict=True,
        reasoning_effort="medium",
    ).to(model.device)

    generated = model.generate(**inputs, max_new_tokens=1000)
    response = tokenizer.decode(
        generated[0][inputs["input_ids"].shape[-1]:],
        skip_special_tokens=True
    )
    print("Response:", response)

    predicted_output = extract_answer_from_response(response)
    expected_output = sample['output'][case_index]
    print("Predicted output:", predicted_output)
    print("Expected output:", expected_output)

    is_correct, debug = check_predicted_output(predicted_output, expected_output)
    print("Correct?", is_correct)
    if debug:
        print("Details:", debug)

    mutated_prompt = build_execution_prediction_prompt(sample, use_mutated=True, case_index=case_index)
    print("Mutated prompt:", mutated_prompt)

    mutated_inputs = tokenizer.apply_chat_template(
        [{"role": "user", "content": mutated_prompt}],
        add_generation_prompt=True,
        return_tensors="pt",
        return_dict=True,
        reasoning_effort="medium",
    ).to(model.device)

    mutated_generated = model.generate(**mutated_inputs, max_new_tokens=1000)
    mutated_response = tokenizer.decode(
        mutated_generated[0][mutated_inputs["input_ids"].shape[-1]:],
        skip_special_tokens=True
    )
    print("Mutated response:", mutated_response)

    mutated_predicted_output = extract_answer_from_response(mutated_response)
    expected_mutated_output = sample['mutated_output'][case_index]
    print("Mutated predicted output:", mutated_predicted_output)
    print("Mutated expected output:", expected_mutated_output)

    is_mutated_correct, debug_mutated = check_predicted_output(
        mutated_predicted_output, expected_mutated_output
    )
    print("Mutated correct?", is_mutated_correct)
    if debug_mutated:
        print("Mutated details:", debug_mutated)


Evaluating sample: dsl_neural_0
=== Case 1 ===
Prompt: You are given a Python program and an assertion containing an input to a function. Replace the ?? in the assertion with a literal (no unsimplified expressions, no function calls) representing the function's return value for the given input. Execute the program exactly as written, even if it is incorrect or incomplete. For your final answer, provide the full assertion in [ANSWER] and [/ANSWER] tags.

[PYTHON]
def dsl_prog_0(a1):
    return (a1)[1:]

assert dsl_prog_0(a1=[0, 0, 0, 5]) == ??
[/PYTHON]
Response: analysisWe need to evaluate the function: def dsl_prog_0(a1): return (a1)[1:]; So we trim the first element of the list [0,0,0,5], yielding [0,0,5]. Actually slicing [1:] returns all elements from index 1 onward. Given [0,0,0,5], slice from index 1 yields [0,0,5]. Wait original list: index 0:0, index1:0, index2:0, index3:5. So slice [1:] yields elements indices 1,2,3: [0,0,5]. So the returned value is [0,0,5]. So the assertion 

## Step 6: Benchmark Execution Prediction on DSL Dataset (OC / OR / MC / MR)

Generate multiple samples per problem to estimate pass@1 for original/mutated correctness and reversion metrics.

In [ ]:
import random
import time
from typing import Optional

import pandas as pd
from tqdm.auto import tqdm

NUM_PROBLEMS = None            # Set to None to evaluate the full dataset
START_INDEX = 0
NUM_GENERATIONS = 5
REASONING_EFFORT = "low"
MAX_NEW_TOKENS = 1000
TEMPERATURE = 0.6
TOP_P = 0.95
SEED = 42
SKIP_BOOLEAN_FOR_REVERSION = True

TEST_CASES_PER_PROBLEM = len(dataset['train'][0]['input'])

torch.manual_seed(SEED)
random.seed(SEED)

pad_token_id = tokenizer.eos_token_id if tokenizer.eos_token_id is not None else tokenizer.pad_token_id


def generate_prediction(prompt: str, seed: int) -> dict:
    generator = torch.Generator(device=model.device)
    generator.manual_seed(seed)
    print("prompt: ", prompt)
    messages = [{"role": "user", "content": prompt}]
    inputs = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        return_tensors="pt",
        return_dict=True,
        reasoning_effort=REASONING_EFFORT,
    ).to(model.device)

    start_time = time.time()
    generation_kwargs = {
        "max_new_tokens": MAX_NEW_TOKENS,
        "do_sample": True,
        "temperature": TEMPERATURE,
        "top_p": TOP_P,
    }
    if pad_token_id is not None:
        generation_kwargs["pad_token_id"] = pad_token_id

    output_ids = model.generate(**inputs, **generation_kwargs)
    latency = time.time() - start_time

    response = tokenizer.decode(
        output_ids[0][inputs["input_ids"].shape[-1]:],
        skip_special_tokens=True,
    )
    print("response:, ", response)
    prediction = extract_answer_from_response(response)

    return {
        "prediction": prediction,
        "response": response,
        "latency_s": latency,
    }


if NUM_PROBLEMS is None:
    problem_indices = list(range(len(dataset["train"])))
else:
    stop = min(len(dataset["train"]), START_INDEX + NUM_PROBLEMS)
    problem_indices = list(range(START_INDEX, stop))

if not problem_indices:
    raise ValueError("No problems selected for evaluation. Adjust START_INDEX/NUM_PROBLEMS.")

print(
    f"Evaluating {len(problem_indices)} DSL problems with {NUM_GENERATIONS} generations "
    f"per test case (x {TEST_CASES_PER_PROBLEM} cases per problem)."
)
print(f"Reasoning effort: {REASONING_EFFORT}, temperature: {TEMPERATURE}, top_p: {TOP_P}")

metrics_counts = {
    "OC": {"success": 0, "total": 0},
    "OR": {"success": 0, "total": 0},
    "MC": {"success": 0, "total": 0},
    "MR": {"success": 0, "total": 0},
}

reversion_skip_count = 0
all_latencies = []
results = []


for idx in tqdm(problem_indices, desc="Evaluating problems"):
    sample = dataset["train"][idx]

    original_outputs = _get_output_list(sample, "output")
    mutated_outputs = _get_output_list(sample, "mutated_output")

    for case_index in range(TEST_CASES_PER_PROBLEM):
        original_prompt = build_execution_prediction_prompt(sample, use_mutated=False, case_index=case_index)
        mutated_prompt = build_execution_prediction_prompt(sample, use_mutated=True, case_index=case_index)

        original_output = original_outputs[case_index]
        mutated_output = mutated_outputs[case_index]

        include_reversion = True
        if SKIP_BOOLEAN_FOR_REVERSION and (is_boolean_output(original_output) or is_boolean_output(mutated_output)):
            include_reversion = False
            reversion_skip_count += 1

        oc_successes = 0
        or_successes = 0
        mc_successes = 0
        mr_successes = 0

        original_predictions = []
        mutated_predictions = []

        seed_base = SEED + idx * 1000 + case_index * 100

        for gen_idx in range(NUM_GENERATIONS):
            pred_original = generate_prediction(original_prompt, seed_base + gen_idx)
            original_predictions.append(pred_original)
            all_latencies.append(pred_original["latency_s"])

            is_correct, _ = check_predicted_output(pred_original["prediction"], original_output)
            if is_correct:
                oc_successes += 1

            if include_reversion:
                is_reversion, _ = check_predicted_output(pred_original["prediction"], mutated_output)
                if is_reversion:
                    or_successes += 1

            pred_mutated = generate_prediction(mutated_prompt, seed_base + 500 + gen_idx)
            mutated_predictions.append(pred_mutated)
            all_latencies.append(pred_mutated["latency_s"])

            is_mutated_correct, _ = check_predicted_output(pred_mutated["prediction"], mutated_output)
            if is_mutated_correct:
                mc_successes += 1

            if include_reversion:
                is_mutated_reversion, _ = check_predicted_output(pred_mutated["prediction"], original_output)
                if is_mutated_reversion:
                    mr_successes += 1

        metrics_counts["OC"]["success"] += oc_successes
        metrics_counts["OC"]["total"] += NUM_GENERATIONS

        metrics_counts["MC"]["success"] += mc_successes
        metrics_counts["MC"]["total"] += NUM_GENERATIONS

        if include_reversion:
            metrics_counts["OR"]["success"] += or_successes
            metrics_counts["OR"]["total"] += NUM_GENERATIONS

            metrics_counts["MR"]["success"] += mr_successes
            metrics_counts["MR"]["total"] += NUM_GENERATIONS

        results.append({
            "problem_index": int(idx),
            "problem_id": sample["id"],
            "case_index": case_index,
            "function_name": sample["function_name"],
            "include_reversion": include_reversion,
            "original_output": original_output,
            "mutated_output": mutated_output,
            "oc_successes": oc_successes,
            "or_successes": or_successes if include_reversion else None,
            "mc_successes": mc_successes,
            "mr_successes": mr_successes if include_reversion else None,
            "original_predictions": original_predictions,
            "mutated_predictions": mutated_predictions,
        })


def compute_pass(counts: dict) -> Optional[float]:
    total = counts["total"]
    if total == 0:
        return None
    return counts["success"] / total


metrics_summary = {metric: compute_pass(counts) for metric, counts in metrics_counts.items()}

total_cases_evaluated = len(problem_indices) * TEST_CASES_PER_PROBLEM

benchmark_summary = {
    "dataset": "DSL-List",
    "problems_evaluated": len(problem_indices),
    "cases_per_problem": TEST_CASES_PER_PROBLEM,
    "generations_per_case": NUM_GENERATIONS,
    "oc_pass_at_1": metrics_summary["OC"],
    "or_pass_at_1": metrics_summary["OR"],
    "mc_pass_at_1": metrics_summary["MC"],
    "mr_pass_at_1": metrics_summary["MR"],
    "avg_latency_s": (sum(all_latencies) / len(all_latencies)) if all_latencies else None,
    "reversion_skipped_cases": reversion_skip_count if SKIP_BOOLEAN_FOR_REVERSION else 0,
}

benchmark_table = pd.DataFrame([{
    "Dataset": "DSL-List",
    "Problems Evaluated": benchmark_summary["problems_evaluated"],
    "Cases per Problem": benchmark_summary["cases_per_problem"],
    "Generations per Case": benchmark_summary["generations_per_case"],
    "OC pass@1": benchmark_summary["oc_pass_at_1"],
    "OR pass@1": benchmark_summary["or_pass_at_1"],
    "MC pass@1": benchmark_summary["mc_pass_at_1"],
    "MR pass@1": benchmark_summary["mr_pass_at_1"],
}])

formatters = {
    "OC pass@1": lambda v, _pd=pd: 'N/A' if _pd.isna(v) else f"{v:.2%}",
    "OR pass@1": lambda v, _pd=pd: 'N/A' if _pd.isna(v) else f"{v:.2%}",
    "MC pass@1": lambda v, _pd=pd: 'N/A' if _pd.isna(v) else f"{v:.2%}",
    "MR pass@1": lambda v, _pd=pd: 'N/A' if _pd.isna(v) else f"{v:.2%}",
}

display(benchmark_table.style.format(formatters))

print("Execution Prediction Summary:")
print(json.dumps(benchmark_summary, indent=2))


Evaluating 100 DSL problems with 5 generations per test case (x 3 cases per problem).
Reasoning effort: low, temperature: 0.6, top_p: 0.95


Evaluating problems:   0%|          | 0/100 [00:00<?, ?it/s]

Streaming output truncated to the last 5000 lines.
prompt:  You are given a Python program and an assertion containing an input to a function. Replace the ?? in the assertion with a literal (no unsimplified expressions, no function calls) representing the function's return value for the given input. Execute the program exactly as written, even if it is incorrect or incomplete. For your final answer, provide the full assertion in [ANSWER] and [/ANSWER] tags.

[PYTHON]
def dsl_prog_105(a1):
    return a1 + [5]
assert dsl_prog_105(a1=[3, 0, 3]) == ??
[/PYTHON]
response:,  analysisCompute: a1=[3,0,3]; return a1 + [5] -> [3,0,3,5]. So assertion: == [3,0,3,5].assistantfinal[ANSWER] assert dsl_prog_105(a1=[3, 0, 3]) == [3, 0, 3, 5] [/ANSWER]
prompt:  You are given a Python program and an assertion containing an input to a function. Replace the ?? in the assertion with a literal (no unsimplified expressions, no function calls) representing the function's return value for the given input. Execu

,Dataset,Problems Evaluated,Cases per Problem,Generations per Case,OC pass@1,OR pass@1,MC pass@1,MR pass@1
0,DSL-List,100,3,5,95.20%,0.00%,96.33%,0.07%


Execution Prediction Summary:
{
  "dataset": "DSL-List",
  "problems_evaluated": 100,
  "cases_per_problem": 3,
  "generations_per_case": 5,
  "oc_pass_at_1": 0.952,
  "or_pass_at_1": 0.0,
  "mc_pass_at_1": 0.9633333333333334,
  "mr_pass_at_1": 0.0006666666666666666,
  "avg_latency_s": 11.479277408202488,
  "reversion_skipped_cases": 0
}


## Step 7: Save Results

In [ ]:
# Plot execution prediction metrics
benchmark_table


,Dataset,Problems Evaluated,Cases per Problem,Generations per Case,OC pass@1,OR pass@1,MC pass@1,MR pass@1
0,DSL-List,100,3,5,0.952,0.0,0.963333,0.000667


In [ ]:
import json
import math
from datetime import datetime

if 'benchmark_summary' not in globals():
    raise RuntimeError('Run the benchmark cell first to produce metrics.')

timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
output_filename = f"gpt_oss_20b_dsl_benchmark_{timestamp}.json"


def _clean_nan(value):
    if value is None:
        return None
    if isinstance(value, float) and math.isnan(value):
        return None
    return value


payload = {
    "model": "gpt-oss-20b",
    "dataset": benchmark_summary["dataset"],
    "reasoning_effort": REASONING_EFFORT,
    "num_problems": benchmark_summary["problems_evaluated"],
    "num_generations": benchmark_summary["generations_per_case"],
    "temperature": TEMPERATURE,
    "top_p": TOP_P,
    "seed": SEED,
    "skip_boolean_for_reversion": SKIP_BOOLEAN_FOR_REVERSION,
    "reversion_skipped_problems": benchmark_summary["reversion_skipped_cases"],
    "metrics": {k: _clean_nan(v) for k, v in metrics_summary.items()},
    "metrics_counts": metrics_counts,
    "benchmark_summary": {k: _clean_nan(v) for k, v in benchmark_summary.items()},
    "results": results,
}

if 'execution_choice_summary' in globals():
    payload['execution_choice_summary'] = {k: _clean_nan(v) for k, v in execution_choice_summary.items()}
    payload['execution_choice_counts'] = execution_choice_counts
    payload['execution_choice_results'] = execution_choice_results
    payload['execution_choice_config'] = {
        'num_problems': NUM_PROBLEMS_CHOICE,
        'start_index': START_INDEX_CHOICE,
        'runs_per_problem': NUM_RUNS_PER_PROBLEM,
        'reasoning_effort': REASONING_EFFORT_CHOICE,
        'max_new_tokens': MAX_NEW_TOKENS_CHOICE,
        'temperature': TEMPERATURE_CHOICE,
        'top_p': TOP_P_CHOICE,
        'seed': SEED_CHOICE,
        'skip_boolean_for_reversion': SKIP_BOOLEAN_FOR_REVERSION_CHOICE,
    }

with open(output_filename, "w") as f:
    json.dump(payload, f, indent=2)

print(f"✓ Saved evaluation summary to {output_filename}")

try:
    from google.colab import files
    files.download(output_filename)
except ImportError:
    print("(Optional) Run inside Colab to download the file automatically.")


✓ Saved evaluation summary to gpt_oss_20b_leetcode_benchmark_20251128_053510.json


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Step 8: Test Execution Choice on One Sample

This cell demonstrates the **Execution Choice** task:

- Model receives two Python programs (original and mutated) and an assertion with `??`
- Model chooses which program (A or B) it wants to execute
- Model predicts what the output will be for the chosen program
- Model returns its choice and the completed assertion in a JSON dictionary


In [ ]:
SKIP_BOOLEAN_FOR_REVERSION_CHOICE = True


In [ ]:
"""## Optional: Test Execution Choice on One Sample

This cell demonstrates the **Execution Choice** task on a single sample:
1. Model sees two programs (original and mutated) plus an assertion with `??`
2. Model chooses which program (A or B) it wants to execute
3. Model fills in the assertion
4. We compute OC / OR / MC / MR metrics for this one sample.
"""

sample = dataset['train'][0]

print(f"Execution Choice sample: {sample['id']}")
print(f"Function name: {sample['function_name']}")

single_choice_counts = {
    "OC": {"correct": 0, "total": 0},
    "OR": {"correct": 0, "total": 0},
    "MC": {"correct": 0, "total": 0},
    "MR": {"correct": 0, "total": 0},
}

for case_index in range(len(sample['input'])):
    print(f"\n=== Execution Choice Case {case_index + 1} ===")
    print(f"Input: {sample['input'][case_index]}")
    print(f"Original output: {sample['output'][case_index]}")
    print(f"Mutated output:  {sample['mutated_output'][case_index]}")

    original_output = sample['output'][case_index]
    mutated_output = sample['mutated_output'][case_index]

    include_reversion = True
    if SKIP_BOOLEAN_FOR_REVERSION_CHOICE and (
        is_boolean_output(original_output) or is_boolean_output(mutated_output)
    ):
        include_reversion = False
        print("Skipping reversion metrics for this case (boolean output).")

    for original_first in (True, False):
        order_label = "Original first (A=original, B=mutated)" if original_first else "Mutated first (A=mutated, B=original)"
        print(f"\n--- {order_label} ---")

        prompt, mapping = build_execution_choice_prompt(sample, original_first, case_index)
        print("Prompt:", prompt)

        messages = [{"role": "user", "content": prompt}]
        inputs = tokenizer.apply_chat_template(
            messages,
            add_generation_prompt=True,
            return_tensors="pt",
            return_dict=True,
            reasoning_effort="medium",
        ).to(model.device)

        generated = model.generate(**inputs, max_new_tokens=5000)
        response = tokenizer.decode(
            generated[0][inputs["input_ids"].shape[-1]:],
            skip_special_tokens=True
        )
        print("Response:", response)

        try:
            parsed = parse_execution_choice_response(response)
        except ValueError as e:
            print("Could not parse execution choice response:", e)
            continue

        chosen_label = parsed.get("chosen_program")
        assertion = parsed.get("assertion")

        print("Parsed chosen_program:", chosen_label)
        print("Parsed assertion:", assertion)

        if chosen_label not in mapping:
            print("Chosen label not in mapping; skipping.")
            continue

        chosen_variant = mapping[chosen_label]  # 'original' or 'mutated'
        predicted_output = extract_output_from_assertion(assertion)

        target_output = original_output if chosen_variant == 'original' else mutated_output
        other_output = mutated_output if chosen_variant == 'original' else original_output

        is_correct, debug_correct = check_predicted_output(predicted_output, target_output)
        is_reversion, debug_reversion = check_predicted_output(predicted_output, other_output)

        print(f"Chosen variant: {chosen_variant}")
        print(f"Predicted output: {predicted_output}")
        print(f"Correct for chosen program? {is_correct}")
        if debug_correct:
            print("  Details (correctness):", debug_correct)
        print(f"Matches other program output (reversion)? {is_reversion}")
        if debug_reversion:
            print("  Details (reversion):", debug_reversion)

        if chosen_variant == 'original':
            single_choice_counts["OC"]["total"] += 1
            if is_correct:
                single_choice_counts["OC"]["correct"] += 1

            if include_reversion:
                single_choice_counts["OR"]["total"] += 1
                if is_reversion:
                    single_choice_counts["OR"]["correct"] += 1
        else:
            single_choice_counts["MC"]["total"] += 1
            if is_correct:
                single_choice_counts["MC"]["correct"] += 1

            if include_reversion:
                single_choice_counts["MR"]["total"] += 1
                if is_reversion:
                    single_choice_counts["MR"]["correct"] += 1

def _single_safe_ratio(num, denom):
    return None if denom == 0 else num / denom

print("\n=== Single-sample Execution Choice Metrics (OC / OR / MC / MR) ===")
for metric in ["OC", "OR", "MC", "MR"]:
    total = single_choice_counts[metric]["total"]
    correct = single_choice_counts[metric]["correct"]
    rate = _single_safe_ratio(correct, total)
    print(
        f"{metric}: correct={correct}, total={total}, "
        f"rate={(f'{rate:.2%}' if rate is not None else 'N/A')}"
    )


Execution Choice sample: dsl_neural_0
Function name: dsl_prog_0

=== Execution Choice Case 1 ===
Input: dsl_prog_0(a1=[0, 0, 0, 5])
Original output: [0, 0, 5]
Mutated output:  [0, 5]

--- Original first (A=original, B=mutated) ---
Prompt: You are given two Python programs below and an assertion containing an input to a function. First, choose either program, whichever one you are more confident in reasoning about. Then, replace the ?? in the assertion with a literal (no unsimplified expressions, no function calls) representing the function's return value for the given input on your chosen program. Execute the program exactly as written, even if it is incorrect or incomplete. For your final answer, output the letter of your chosen program (A or B) and the full assertion in the following json format:

{{
  "chosen_program": "A or B",
  "assertion": "full_assertion"
}}

[PROGRAM_A]
def dsl_prog_0(a1):
    return (a1)[1:]

[/PROGRAM_A]
[PROGRAM_B]
def dsl_prog_0(a1):
    return a1[2:]
[/PR

## Step 9: Execution Choice Benchmark on DSL Dataset (Preference / Correctness / Reversion)

Run the paired-program experiment to measure program preference, correctness, and reversion with order swapping.

In [ ]:
import random
import time
from typing import Optional

import pandas as pd
from tqdm.auto import tqdm


# Ensure NUM_PROBLEMS exists even if Step 6 wasn't executed
try:
    NUM_PROBLEMS
except NameError:
    NUM_PROBLEMS = None

try:
    START_INDEX
except NameError:
    START_INDEX = 0
TEST_CASES_PER_PROBLEM = len(dataset['train'][0]['input'])

NUM_RUNS_PER_PROBLEM = 1
REASONING_EFFORT_CHOICE = "low"
MAX_NEW_TOKENS_CHOICE = 5000
TEMPERATURE_CHOICE = 0.6
TOP_P_CHOICE = 0.95
SEED_CHOICE = 123
SKIP_BOOLEAN_FOR_REVERSION_CHOICE = True

random.seed(SEED_CHOICE)
torch.manual_seed(SEED_CHOICE)

pad_token_choice = tokenizer.eos_token_id if tokenizer.eos_token_id is not None else tokenizer.pad_token_id


def generate_choice(prompt: str, seed: int) -> dict:
    generator = torch.Generator(device=model.device)
    generator.manual_seed(seed)
    messages = [{"role": "user", "content": prompt}]
    inputs = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        return_tensors="pt",
        return_dict=True,
        reasoning_effort=REASONING_EFFORT_CHOICE,
    ).to(model.device)

    start_time = time.time()
    generation_kwargs = {
        "max_new_tokens": MAX_NEW_TOKENS_CHOICE,
        "do_sample": True,
        "temperature": TEMPERATURE_CHOICE,
        "top_p": TOP_P_CHOICE,
    }
    if pad_token_choice is not None:
        generation_kwargs["pad_token_id"] = pad_token_choice

    output_ids = model.generate(**inputs, **generation_kwargs)
    latency = time.time() - start_time

    response = tokenizer.decode(
        output_ids[0][inputs["input_ids"].shape[-1]:],
        skip_special_tokens=True,
    )

    return {
        "response": response,
        "latency_s": latency,
    }


if NUM_PROBLEMS is None:
    choice_problem_indices = list(range(len(dataset["train"])))
else:
    stop = min(len(dataset["train"]), START_INDEX + NUM_PROBLEMS)
    choice_problem_indices = list(range(START_INDEX, stop))

print(
    f"Running execution choice on {len(choice_problem_indices)} DSL problems "
    f"with {NUM_RUNS_PER_PROBLEM} runs per case."
)

execution_choice_counts = {
    'preference': {'original': 0, 'mutated': 0},
    'OC': {'correct': 0, 'total': 0},
    'OR': {'correct': 0, 'total': 0},
    'MC': {'correct': 0, 'total': 0},
    'MR': {'correct': 0, 'total': 0, 'reversion_correct': 0, 'reversion_total': 0},
    'invalid_runs': 0,
}

execution_choice_latencies = []
reversion_skip_count_choice = 0

for idx in tqdm(choice_problem_indices, desc="Execution choice problems"):
    sample = dataset['train'][idx]

    original_outputs = _get_output_list(sample, 'output')
    mutated_outputs = _get_output_list(sample, 'mutated_output')

    for case_index in range(TEST_CASES_PER_PROBLEM):
        prompt_original_first, mapping_original_first = build_execution_choice_prompt(sample, True, case_index)
        prompt_mutated_first, mapping_mutated_first = build_execution_choice_prompt(sample, False, case_index)

        original_output = original_outputs[case_index]
        mutated_output = mutated_outputs[case_index]

        include_reversion = True
        if SKIP_BOOLEAN_FOR_REVERSION_CHOICE and (
            is_boolean_output(original_output) or is_boolean_output(mutated_output)
        ):
            include_reversion = False
            reversion_skip_count_choice += 1

        seed_base = SEED_CHOICE + idx * 1000 + case_index * 100

        for run_idx in range(NUM_RUNS_PER_PROBLEM):
            for prompt, mapping in (
                (prompt_original_first, mapping_original_first),
                (prompt_mutated_first, mapping_mutated_first),
            ):
                generation = generate_choice(prompt, seed_base + run_idx)
                execution_choice_latencies.append(generation["latency_s"])

                response = generation["response"]
                try:
                    parsed = parse_execution_choice_response(response)
                except ValueError:
                    execution_choice_counts['invalid_runs'] += 1
                    continue

                chosen_label = parsed.get('chosen_program')
                assertion = parsed.get('assertion')
                if chosen_label not in mapping:
                    execution_choice_counts['invalid_runs'] += 1
                    continue

                chosen_variant = mapping[chosen_label]
                predicted_output = extract_output_from_assertion(assertion)

                target_output = original_output if chosen_variant == 'original' else mutated_output
                other_output = mutated_output if chosen_variant == 'original' else original_output

                is_correct, _ = check_predicted_output(predicted_output, target_output)
                is_reversion, _ = check_predicted_output(predicted_output, other_output)

                if chosen_variant == 'original':
                    execution_choice_counts['preference']['original'] += 1
                else:
                    execution_choice_counts['preference']['mutated'] += 1

                if chosen_variant == 'original':
                    execution_choice_counts['OC']['total'] += 1
                    if is_correct:
                        execution_choice_counts['OC']['correct'] += 1
                    if include_reversion:
                        execution_choice_counts['OR']['total'] += 1
                        if is_reversion:
                            execution_choice_counts['OR']['correct'] += 1
                else:
                    execution_choice_counts['MC']['total'] += 1
                    if is_correct:
                        execution_choice_counts['MC']['correct'] += 1
                    if include_reversion:
                        execution_choice_counts['MR']['reversion_total'] += 1
                        if is_reversion:
                            execution_choice_counts['MR']['reversion_correct'] += 1


preference_total = execution_choice_counts['preference']['original'] + execution_choice_counts['preference']['mutated']
preference_original_rate = execution_choice_counts['preference']['original'] / preference_total if preference_total else None
preference_mutated_rate = execution_choice_counts['preference']['mutated'] / preference_total if preference_total else None

def _safe_ratio(num, denom):
    return None if denom == 0 else num / denom

oc_correct_rate = _safe_ratio(execution_choice_counts['OC']['correct'], execution_choice_counts['OC']['total'])
or_reversion_rate = _safe_ratio(execution_choice_counts['OR']['correct'], execution_choice_counts['OR']['total'])
mc_correct_rate = _safe_ratio(execution_choice_counts['MC']['correct'], execution_choice_counts['MC']['total'])
mr_reversion_rate = _safe_ratio(
    execution_choice_counts['MR']['reversion_correct'], execution_choice_counts['MR']['reversion_total']
)

execution_choice_summary = {
    'dataset': 'DSL-List',
    'problems_evaluated': len(choice_problem_indices),
    'cases_per_problem': TEST_CASES_PER_PROBLEM,
    'runs_per_case': NUM_RUNS_PER_PROBLEM,
    'preference_original': preference_original_rate,
    'preference_mutated': preference_mutated_rate,
    'oc_correct': oc_correct_rate,
    'or_reversion': or_reversion_rate,
    'mc_correct': mc_correct_rate,
    'mr_reversion': mr_reversion_rate,
    'avg_latency_s': (
        sum(execution_choice_latencies) / len(execution_choice_latencies)
        if execution_choice_latencies else None
    ),
    'invalid_runs': execution_choice_counts['invalid_runs'],
    'reversion_skipped_cases': (
        reversion_skip_count_choice if SKIP_BOOLEAN_FOR_REVERSION_CHOICE else 0
    ),
}

execution_choice_table = pd.DataFrame([{
    'Dataset': 'DSL-List',
    'Problems Evaluated': execution_choice_summary['problems_evaluated'],
    'Cases per Problem': execution_choice_summary['cases_per_problem'],
    'Runs per Case': execution_choice_summary['runs_per_case'],
    'Preference (Original)': execution_choice_summary['preference_original'],
    'OC Correct': execution_choice_summary['oc_correct'],
    'OR Reversion': execution_choice_summary['or_reversion'],
    'MC Correct': execution_choice_summary['mc_correct'],
    'MR Reversion': execution_choice_summary['mr_reversion'],
}])

choice_formatters = {
    'Preference (Original)': lambda v, _pd=pd: 'N/A' if _pd.isna(v) else f"{v:.2%}",
    'OC Correct': lambda v, _pd=pd: 'N/A' if _pd.isna(v) else f"{v:.2%}",
    'OR Reversion': lambda v, _pd=pd: 'N/A' if _pd.isna(v) else f"{v:.2%}",
    'MC Correct': lambda v, _pd=pd: 'N/A' if _pd.isna(v) else f"{v:.2%}",
    'MR Reversion': lambda v, _pd=pd: 'N/A' if _pd.isna(v) else f"{v:.2%}",
}

display(execution_choice_table.style.format(choice_formatters))

print('Preference counts:')
print(
    f"  Original: {execution_choice_counts['preference']['original']} / {preference_total}"
)
print(
    f"  Mutated: {execution_choice_counts['preference']['mutated']} / {preference_total}"
)
print(f"Invalid runs (no usable JSON response): {execution_choice_counts['invalid_runs']}")
if execution_choice_summary['avg_latency_s'] is not None:
    print(f"Average latency per run: {execution_choice_summary['avg_latency_s']:.2f}s")
else:
    print('Average latency per run: N/A')
if SKIP_BOOLEAN_FOR_REVERSION_CHOICE:
    print(
        f"Cases skipped for reversion metrics (boolean outputs): {reversion_skip_count_choice}"
    )


Running execution choice on 100 DSL problems with 1 runs per case.


Execution choice problems:   0%|          | 0/100 [00:00<?, ?it/s]

,Dataset,Problems Evaluated,Cases per Problem,Runs per Case,Preference (Original),OC Correct,OR Reversion,MC Correct,MR Reversion
0,DSL-List,100,3,1,52.17%,95.85%,2.09%,94.77%,0.00%


Preference counts:
  Original: 313 / 600
  Mutated: 287 / 600
Invalid runs (no usable JSON response): 0
Average latency per run: 24.18s
Cases skipped for reversion metrics (boolean outputs): 0


In [ ]:
import json

# Display execution choice metrics
execution_choice_table

print("Execution Choice Summary:")
print(json.dumps(execution_choice_summary, indent=2))


Execution Choice Summary:
{
  "dataset": "DSL-List",
  "problems_evaluated": 100,
  "cases_per_problem": 3,
  "runs_per_case": 1,
  "preference_original": 0.5216666666666666,
  "preference_mutated": 0.47833333333333333,
  "oc_correct": 0.9584664536741214,
  "or_reversion": 0.020905923344947737,
  "mc_correct": 0.9477351916376306,
  "mr_reversion": 0.0,
  "avg_latency_s": 24.175232906738916,
  "invalid_runs": 0,
  "reversion_skipped_cases": 0
}


In [ ]:
"""## Step 9: Save Execution Choice Results"""

import json
import math
from datetime import datetime

if 'execution_choice_summary' not in globals():
    raise RuntimeError('Run the execution choice benchmark cell first to produce metrics.')

timestamp_choice = datetime.now().strftime("%Y%m%d_%H%M%S")
output_filename_choice = f"gpt_oss_20b_dsl_execution_choice_{timestamp_choice}.json"

def _clean_nan_choice(value):
    if value is None:
        return None
    if isinstance(value, float) and math.isnan(value):
        return None
    return value

execution_choice_payload = {
    "model": "gpt-oss-20b",
    "dataset": execution_choice_summary["dataset"],
    "reasoning_effort": REASONING_EFFORT_CHOICE,
    "num_problems": execution_choice_summary["problems_evaluated"],
    "runs_per_case": execution_choice_summary["runs_per_case"],
    "temperature": TEMPERATURE_CHOICE,
    "top_p": TOP_P_CHOICE,
    "seed": SEED_CHOICE,
    "skip_boolean_for_reversion": SKIP_BOOLEAN_FOR_REVERSION_CHOICE,
    "reversion_skipped_cases": execution_choice_summary["reversion_skipped_cases"],
    "metrics": {
        "preference_original": _clean_nan_choice(execution_choice_summary["preference_original"]),
        "preference_mutated": _clean_nan_choice(execution_choice_summary["preference_mutated"]),
        "oc_correct": _clean_nan_choice(execution_choice_summary["oc_correct"]),
        "or_reversion": _clean_nan_choice(execution_choice_summary["or_reversion"]),
        "mc_correct": _clean_nan_choice(execution_choice_summary["mc_correct"]),
        "mr_reversion": _clean_nan_choice(execution_choice_summary["mr_reversion"]),
        "avg_latency_s": _clean_nan_choice(execution_choice_summary["avg_latency_s"]),
    },
    "execution_choice_counts": execution_choice_counts,
}

with open(output_filename_choice, "w") as f:
    json.dump(execution_choice_payload, f, indent=2)

print(f"✓ Saved execution choice summary to {output_filename_choice}")

try:
    from google.colab import files
    files.download(output_filename_choice)
except ImportError:
    print("(Optional) Run inside Colab to download the file automatically.")


✓ Saved execution choice summary to gpt_oss_20b_dsl_execution_choice_20251130_002233.json


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Step 10: Save Execution Choice Results

In [ ]:
"""## [SIMILAR FORMAT] Step 9: Save Execution Choice Results"""

import json
import math
from datetime import datetime

if 'execution_choice_summary' not in globals():
    raise RuntimeError('Run the execution choice benchmark cell first to produce metrics.')

timestamp_choice = datetime.now().strftime("%Y%m%d_%H%M%S")
output_filename_choice = f"gpt_oss_20b_dsl_execution_choice_{timestamp_choice}.json"

# Build metrics dict in the same style as execution prediction
execution_choice_metrics = {
    "OC": execution_choice_summary["oc_correct"],
    "OR": execution_choice_summary["or_reversion"],
    "MC": execution_choice_summary["mc_correct"],
    "MR": execution_choice_summary["mr_reversion"],
}

# Build metrics_counts dict similar to metrics_counts in prediction JSON
execution_choice_metrics_counts = {
    "OC": {
        "success": execution_choice_counts["OC"]["correct"],
        "total": execution_choice_counts["OC"]["total"],
    },
    "OR": {
        "success": execution_choice_counts["OR"]["correct"],
        "total": execution_choice_counts["OR"]["total"],
    },
    "MC": {
        "success": execution_choice_counts["MC"]["correct"],
        "total": execution_choice_counts["MC"]["total"],
    },
    "MR": {
        "success": execution_choice_counts["MR"]["reversion_correct"],
        "total": execution_choice_counts["MR"]["reversion_total"],
    },
}

benchmark_summary_choice = {
    "dataset": execution_choice_summary["dataset"],
    "problems_evaluated": execution_choice_summary["problems_evaluated"],
    "cases_per_problem": execution_choice_summary["cases_per_problem"],
    "generations_per_case": execution_choice_summary["runs_per_case"],
    "oc_pass_at_1": execution_choice_summary["oc_correct"],
    "or_pass_at_1": execution_choice_summary["or_reversion"],
    "mc_pass_at_1": execution_choice_summary["mc_correct"],
    "mr_pass_at_1": execution_choice_summary["mr_reversion"],
    "avg_latency_s": execution_choice_summary["avg_latency_s"],
    "reversion_skipped_cases": execution_choice_summary["reversion_skipped_cases"],
    # extra choice-specific stats (fine to have more than prediction JSON)
    "preference_original": execution_choice_summary["preference_original"],
    "preference_mutated": execution_choice_summary["preference_mutated"],
    "invalid_runs": execution_choice_summary["invalid_runs"],
}

def _clean_nan_choice(value):
    if value is None:
        return None
    if isinstance(value, float) and math.isnan(value):
        return None
    return value

execution_choice_payload = {
    "model": "gpt-oss-20b",
    "dataset": execution_choice_summary["dataset"],
    "reasoning_effort": REASONING_EFFORT_CHOICE,
    "num_problems": execution_choice_summary["problems_evaluated"],
    "num_generations": execution_choice_summary["runs_per_case"],
    "temperature": TEMPERATURE_CHOICE,
    "top_p": TOP_P_CHOICE,
    "seed": SEED_CHOICE,
    "skip_boolean_for_reversion": SKIP_BOOLEAN_FOR_REVERSION_CHOICE,
    "reversion_skipped_problems": execution_choice_summary["reversion_skipped_cases"],
    "metrics": {k: _clean_nan_choice(v) for k, v in execution_choice_metrics.items()},
    "metrics_counts": {
        k: {
            "success": _clean_nan_choice(v["success"]),
            "total": _clean_nan_choice(v["total"]),
        }
        for k, v in execution_choice_metrics_counts.items()
    },
    "benchmark_summary": {
        k: _clean_nan_choice(v) for k, v in benchmark_summary_choice.items()
    },
}

# Include detailed per-run results if you created them as `execution_choice_results`
if "execution_choice_results" in globals():
    execution_choice_payload["results"] = execution_choice_results
else:
    execution_choice_payload["results"] = []

with open(output_filename_choice, "w") as f:
    json.dump(execution_choice_payload, f, indent=2)

print(f"✓ Saved execution choice summary to {output_filename_choice}")

try:
    from google.colab import files
    files.download(output_filename_choice)
except ImportError:
    print("(Optional) Run inside Colab to download the file automatically.")


✓ Saved execution choice summary to gpt_oss_20b_dsl_execution_choice_20251130_003837.json


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Step 10: Compare Reasoning Efforts (Optional)

Evaluate a small subset with different reasoning effort settings.

In [ ]:
# Compare low vs medium vs high reasoning
reasoning_levels = ["low", "medium", "high"]
comparison_results = {}

NUM_COMPARISON_SAMPLES = min(5, len(dataset['train']))
MAX_NEW_TOKENS = 1000

for reasoning_effort in reasoning_levels:
    print(f"
Testing reasoning_effort={reasoning_effort}...")
    correct = 0
    total_latency = 0

    for idx in range(NUM_COMPARISON_SAMPLES):
        sample = dataset['train'][idx]
        for case_index in range(TEST_CASES_PER_PROBLEM):
            prompt = build_execution_prediction_prompt(sample, case_index=case_index)
            messages = [
                {"role": "user", "content": prompt}
            ]

            inputs = tokenizer.apply_chat_template(
                messages,
                add_generation_prompt=True,
                return_tensors="pt",
                return_dict=True,
                reasoning_effort=reasoning_effort,
            ).to(model.device)

            start_time = time.time()
            generated = model.generate(**inputs, max_new_tokens=MAX_NEW_TOKENS)
            latency = time.time() - start_time
            total_latency += latency

            response = tokenizer.decode(
                generated[0][inputs["input_ids"].shape[-1]:],
                skip_special_tokens=True
            )

            predicted_output = extract_answer_from_response(response)
            expected_output = sample['output'][case_index]

            is_correct, _ = check_predicted_output(predicted_output, expected_output)
            if is_correct:
                correct += 1

    total_cases = NUM_COMPARISON_SAMPLES * TEST_CASES_PER_PROBLEM
    comparison_results[reasoning_effort] = {
        "correct": correct,
        "pass@1": correct / total_cases,
        "avg_latency": total_latency / total_cases
    }

print("
" + "="*60)
print("REASONING EFFORT COMPARISON")
print("="*60)
print(f"{'Reasoning':<12} {'pass@1':<10} {'Avg Latency':<15}")
print("-" * 60)
for level, stats in comparison_results.items():
    print(f"{level:<12} {stats['pass@1']*100:>6.1f}%   {stats['avg_latency']:>10.2f}s")
print("="*60)


## Next Steps

1. **Increase NUM_SAMPLES** to evaluate on more problems
2. **Try different reasoning_effort** levels
3. **Compare with other models** (DeepSeek-R1, GPT-4o, etc.)
4. **Analyze error patterns** to understand model weaknesses

---

**Note**: To run on full dataset (347 samples), expect ~1-2 hours on free Colab.